In [2]:
import os
import pathlib
import pandas as pd
import re

import copy
import numpy as np

from matplotlib import cm
import matplotlib.pyplot as plt

import analysis
%load_ext autoreload
%autoreload 2

# Définir le répertoire racine où vous souhaitez chercher des fichiers .csv
root_directory = os.getcwd() + "/outputs"
# root_directory = os.getcwd() + "/outputs/no_mask/2023-06-14/14-56-43/5"
# Créer une liste vide pour stocker tous les DataFrames des fichiers .csv trouvés
data_frames = []
conv_frames = []
extract_it_and_rec_loss_from_log = False

# Parcourir tous les fichiers du répertoire racine et des sous-répertoires de manière récursive
if_break = False
for subdir, dirs, files in os.walk(root_directory):
    if not "trash" in subdir:
        for file in files:
            # Vérifier si le fichier est un fichier .csv
            if file.endswith('reports.csv'):
                # Ouvrir le fichier .csv dans un DataFrame
                file_path = os.path.join(subdir, file)
                df = pd.read_csv(file_path, sep='\t')
                df["file_path"] = file_path
                # Ajouter le DataFrame à la listeZ

                for subdir_log, _, file_logs in os.walk(pathlib.Path(subdir).parent):
                    for file_log in file_logs:
                        if file_log.endswith('.log'):
                            if file_log.__contains__("paired"):
                                # Extract a, b, c from log file
                                file_path_log = os.path.join(subdir_log, file_log)
                                a, b, c = (analysis.get_paired_from_log(file_path_log))
                                # print(log_file.read())
                                df["num_data_points"] = a
                                df["num_data_per_local_update_step"] = b
                                df["num_local_updates"] = c

                            if file_log.__contains__("load_round"):
                                # Extract load round from log file
                                load_round = analysis.get_load_round_from_log(file_path_log)
                                df["load_round"] = load_round

                            if extract_it_and_rec_loss_from_log: 
                                # Extract Iterations, and Reconstruction Losses from log file
                                user, trials, it_rec_pairs_matches = analysis.get_iteration_and_rec_loss_from_log(file_path_log)
                                second_time = False
                                for it, _ in it_rec_pairs_matches:
                                    df[f"it_{it}"] = None
                                trial = 0
                                max_it = max([it for it, _ in it_rec_pairs_matches])
                                for it, rec_loss in it_rec_pairs_matches:
                                    df.loc[df.index == trial, f"it_{it}"] = rec_loss
                                    if it == max_it:
                                        trial += 1

                            # if_break = True
                            if if_break:
                                break
                    if if_break:
                        break
                data_frames.append(df)
            elif file.endswith('convergence_results.csv'):
                file_path = os.path.join(subdir, file)
                df = pd.read_csv(file_path, sep='\t')
                df["file_path"] = file_path
                conv_frames.append(df)
            if if_break:
                break
        if if_break:
            break

# Fusionner tous les DataFrames en un seul DataFrame
merged_df = pd.concat(data_frames, ignore_index=True)
conv_df = pd.concat(conv_frames, ignore_index=True)
print(merged_df.shape)
print(conv_df.shape)

(3692, 52)
(199200, 5)


In [7]:
print(merged_df.columns)
for col in merged_df.columns:
    print(f"{col:<20}, {merged_df[col].loc[50]}")

Index(['name', 'usecase', 'model', 'datapoints', 'model_state', 'attack',
       'attacktype', 'mse', 'psnr', 'lpips', 'rpsnr', 'ssim', 'min_mse',
       'max_mse', 'max_psnr', 'max_lpips', 'max_ssim', 'max_rpsnr', 'IIP-none',
       'feat_mse', 'parameters', 'label_acc', 'queries', 'score', 'total_time',
       'user_type', 'gradient_noise', 'seed', 'ATK_type', 'ATK_attack_type',
       'ATK_label_strategy', 'ATK_text_strategy', 'ATK_token_recovery',
       'ATK_objective', 'ATK_restarts', 'ATK_init', 'ATK_normalize_gradients',
       'ATK_optim', 'ATK_augmentations', 'ATK_differentiable_augmentations',
       'ATK_regularization', 'ATK_impl', 'data', 'impl', 'server', 'user',
       'num_queries', 'folder', 'file_path', 'IIP-pixel', 'IIP-lpips',
       'IIP-self'],
      dtype='object')
name                , no_mask
usecase             , image_custom
model               , custom_McMahan_32_32
datapoints          , 20
model_state         , default
attack              , deep-leakage
at

In [ ]:
['name', 'usecase', 'model', 'datapoints', 'model_state', 'attack',
       'attacktype', 'mse', 'psnr', 'lpips', 'rpsnr', 'ssim', 'min_mse',
       'max_mse', 'max_psnr', 'max_lpips', 'max_ssim', 'max_rpsnr', 'IIP-none',
       'feat_mse', 'parameters', 'label_acc', 'queries', 'score', 'total_time',
       'user_type', 'gradient_noise', 'seed', 'ATK_type', 'ATK_attack_type',
       'ATK_label_strategy', 'ATK_text_strategy', 'ATK_token_recovery',
       'ATK_objective', 'ATK_restarts', 'ATK_init', 'ATK_normalize_gradients',
       'ATK_optim', 'ATK_augmentations', 'ATK_differentiable_augmentations',
       'ATK_regularization', 'ATK_impl', 'data', 'impl', 'server', 'user',
       'num_queries', 'folder', 'file_path', 'IIP-pixel', 'IIP-lpips',
       'IIP-self']

In [3]:
import wandb
import pandas as pd

PROJECT_NAME = "Global Experiments"

EXPERIMENT_NAME_COL = "name"
# NOTES_COL = "Notes"
# TAGS_COL = "Tags"
CONFIG_COLS = ["name", "usecase", "model", "datapoints", "model_state", 
               "attack", "attacktype", "user_type", "gradient_noise", "seed", 
               "ATK_type", "ATK_attack_type", "ATK_label_strategy", "ATK_text_strategy", 
               "ATK_token_recovery", "ATK_objective", "ATK_restarts", "ATK_init", "ATK_normalize_gradients", 
               "ATK_optim", "ATK_augmentations", "ATK_differentiable_augmentations", "ATK_regularization", 
               "ATK_impl", "data", "impl", "server", "user", "num_queries"]

# SUMMARY_COLS = ["Final Train Acc", "Final Val Acc"]
METRIC_COLS = ["mse", "psnr", "lpips", "rpsnr", "ssim", "min_mse", "max_mse", "max_psnr", "max_lpips", 
               "max_ssim", "max_rpsnr", "IIP-none", "feat_mse", "parameters", "score", "total_time", 
               "IIP-pixel", "IIP-lpips", "IIP-self"]

for i, row in merged_df.iterrows():
    
    run_name = row[EXPERIMENT_NAME_COL]
    # notes = row[NOTES_COL]
    # tags = row[TAGS_COL]

    config = {}
    for config_col in CONFIG_COLS:
        config[config_col] = row[config_col]

    metrics = {}
    for metric_col in METRIC_COLS:
        metrics[metric_col] = row[metric_col]
    
    # summaries = {}
    # for summary_col in SUMMARY_COLS:
    #     summaries[summary_col] = row[summary_col]


run = wandb.init(project=PROJECT_NAME, name=run_name, config=config)

for key, val in metrics.items():
    if isinstance(val, list):
        for _val in val:
            run.log({key: _val})
    else:
        run.log({key: val})
        
# run.summary.update(summaries)
run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pjobic (flprivacy). Use `wandb login --relogin` to force relogin


IIP-lpips,▁
IIP-pixel,▁
IIP-self,▁
feat_mse,▁
lpips,▁
max_rpsnr,▁
max_ssim,▁
mse,▁
parameters,▁
psnr,▁
rpsnr,▁
